In [1]:
import sqlite3
 
class PercentileFunc:
    def __init__(self):
        self.list = []
        self.percent = None
 
    def step(self, value, percent):
        if value is None:
            return
        if self.percent is None:
            self.percent = percent
        if self.percent != percent:
            return
        self.list.append(value)
 
    def finalize(self):
        if len(self.list) == 0:
            return None
        self.list.sort()
        return self.list[int(round((len(self.list)-1)*self.percent/100.0))]
 

## 1. What was the total revenue to the nearest dollar for customers who have paid by credit card?


In [2]:
with sqlite3.connect('test_data.db') as con:    
    cur = con.cursor()
    query1 = f"""
    select round(sum(revenue),0) as tot_rev
    from (
            select customer_id, max(revenue) revenue, max(cc_payments) cc_payments
            from customers 
            group by customer_id
    )cust_tran 
    where cc_payments >0"""
    cur.execute(query1) 
    print("Total revenue for customers who have paid by CC is {}".format(cur.fetchone()[0]))


Total revenue for customers who have paid by CC is 50189329.0


## 2. What percentage of customers who have purchased female items have paid by credit card?

In [ ]:
with sqlite3.connect('test_data.db') as con:    
    cur = con.cursor()
    query2 = f"""
    select round(cc_cust*1.0/all_cust,2) as cc_cust_ratio
    from(
        select count(distinct customer_id) as all_cust
                , sum(case when cc_payments > 0 then 1 else 0 end) as cc_cust
        from (
            select customer_id, sum(female_items) as female_items, sum(cc_payments) as cc_payments
            from customers 
            group by customer_id
        )cust_tran
        where female_items > 0
    )"""
    cur.execute(query2) 
    print("The percentage is {}".format(cur.fetchone()[0]))

## 3. What was the average revenue for customers who used either iOS, Android or Desktop? 

In [ ]:
with sqlite3.connect('test_data.db') as con:    
    cur = con.cursor()
    query3 = f"""
    select round(sum(revenue)/count(customer_id),0) rev_avg
    from(
        select customer_id, max(revenue) as revenue
        from customers 
        group by customer_id
        having sum(desktop_orders) + sum(android_orders) + sum(ios_orders) > 0	
    )"""
    cur.execute(query3) 
    print("The average revenue is {}".format(cur.fetchone()[0]))

## 4. We want to run an email campaign promoting a new mens luxury brand. Can you provide a list of customers we should send to?

In [ ]:
class PercentileFunc:
    def __init__(self):
        self.list = []
        self.percent = None
 
    def step(self, value, percent):
        if value is None:
            return
        if self.percent is None:
            self.percent = percent
        if self.percent != percent:
            return
        self.list.append(value)
 
    def finalize(self):
        if len(self.list) == 0:
            return None
        self.list.sort()
        return self.list[int(round((len(self.list)-1)*self.percent/100.0))]

In [ ]:
#pick up users whose male item is at least x% of total items bought(set1)
#pick up users whose revenue is the top (1-Y% )in terms of income(set2), y is percential value
#inner join set1 and set2
MALE_ITEM_RATIO=0.1
REVENUE_PERCENTILE=55

with sqlite3.connect('test_data.db') as con:    
    cur = con.cursor()
    #generate base table
    query4_0 = f"""
    create table base as
    select customer_id, male_items, female_items, revenue
    from(
        select  customer_id
                , sum(male_items) as male_items
                , sum(female_items) as female_items
                , sum(revenue) revenue
        from customers 
        group by customer_id
    )"""
    cur.execute("drop table if exists base")
    cur.execute(query4_0) 

    
    #majority shopping items are male items
    cur = con.cursor()
    query4_1 = f"""
    create table set1 as
    select customer_id
    from base 
    group by customer_id
    having sum(male_items)*1.0/(sum(male_items) + sum(female_items)) > {MALE_ITEM_RATIO}
    """
    cur.execute("drop table if exists set1")
    cur.execute(query4_1) 

    
    #high revenue customers, first get specifiled percential value
    con.create_aggregate("percentile", 2, PercentileFunc)
    cur.execute(f"""select percentile(revenue, {REVENUE_PERCENTILE}) from base""")
    val = cur.fetchone()[0]
    #high revenue customers, get customers revenue more than specified percentile
    cur = con.cursor()
    query4_2 = f"""
    create table set2 as
    select customer_id
    from base 
    where revenue > {val}
    """
    cur.execute("drop table if exists set2")   
    cur.execute(query4_2) 

    
    #high revewnue customers and majority shopping items are male items
    cur = con.cursor()
    query4_3 = f"""
    create table q4final as
    select set1.customer_id
    from set1
    inner join set2
    on set1.customer_id = set2.customer_id
    """
    cur.execute("drop table if exists q4final")   
    cur.execute(query4_3) 
    
    cur = con.cursor()
    cur.execute("select count(*) from q4final")  
    total_count = cur.fetchone()[0]
    print("As per table query4_3, in output table q4final, there are {} customers selected.".format(total_count))
